# Demo of _narrative graphs_


In [1]:
import kagglehub
import pandas as pd

path = kagglehub.dataset_download("rmisra/news-category-dataset", "News_Category_Dataset_v3.json")
data = pd.read_json(path, lines=True)
data.head()

,link,headline,category,short_description,authors,date
0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",2022-09-23
1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,2022-09-23
2,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,2022-09-23
3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23
4,https://www.huffpost.com/entry/amy-cooper-lose...,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski,2022-09-22


In [2]:
# create a sample
sample = data.sample(2500)
docs = sample["headline"] + "\n" + sample["short_description"]
categories = sample["category"]
timestamps = sample["date"]

In [3]:
from narrativegraph import NarrativeGraph

model = NarrativeGraph()
model.fit(docs, categories=categories, timestamps=timestamps)

INFO:narrativegraph:Adding 2500 documents to database
INFO:narrativegraph:Extracting triplets
Extracting triplets: 100%|██████████| 2500/2500 [00:15<00:00, 156.43it/s]
INFO:narrativegraph:Mapping entities and relations
INFO:narrativegraph:Mapping triplets
Updating relation info: 100%|██████████| 1085/1085 [00:00<00:00, 3771.63it/s]


In [ ]:
# create server to be viewed in own browser which blocks execution of other cells
model.serve_visualizer()

INFO:     Started server process [22595]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8001 (Press CTRL+C to quit)


INFO:     127.0.0.1:49817 - "GET / HTTP/1.1" 307 Temporary Redirect
INFO:     127.0.0.1:49817 - "GET /vis HTTP/1.1" 307 Temporary Redirect
INFO:     127.0.0.1:49817 - "GET /vis/ HTTP/1.1" 304 Not Modified
INFO:     127.0.0.1:49820 - "GET /graph/bounds HTTP/1.1" 200 OK
INFO:     127.0.0.1:49820 - "POST /graph HTTP/1.1" 307 Temporary Redirect
INFO:     127.0.0.1:49820 - "POST /graph/ HTTP/1.1" 200 OK


In [ ]:
# create server to be viewed in this notebook which therefore runs in the background
server = model.serve_visualizer(block=False)

In [ ]:
server.show_iframe()

In [9]:
server.stop()

INFO:root:Background server stopped
